In [97]:
import os
import csv
import json
import logging
import math
from typing import List, Dict, Tuple, Callable, Iterable, Optional, Any

import torch
import torch.nn as nn
import pandas as pd
import pickle
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from numpy import save
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')
folder_ = 'drive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers
!pip install sentence_transformers

In [4]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator, SentenceEvaluator
from sentence_transformers.util import batch_to_device
from torch import Tensor
from torch.utils.data import DataLoader
from transformers import MT5EncoderModel, AutoConfig,T5Tokenizer
from transformers import pipeline, AutoTokenizer

In [5]:
train_df=pd.read_csv(folder_ + 'Data/Train.csv')
#test_df=pd.read_csv('C:/Users/leose/second_semester/NLP/malawi_news_classification/data/test.csv')

In [6]:
train_df['Text']=train_df['Text'].tolist()
#test_df['Text']=test_df['Text'].tolist()
train_batch_sentences=[]
#test_batch_sentences=[]

#for frame in train_df['Text']:
#    train_batch_sentences.append(frame)
#for sentence in test_df['Text']:
#    test_batch_sentences.append(frame)

In [8]:
len(train_df)

1436

In [7]:
model = MT5EncoderModel.from_pretrained("google/mt5-small")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
#article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
#for sentence
#input_ids = tokenizer(train_batch_sentences[0], return_tensors="pt",truncation=True,max_length=512).input_ids
#outputs = model(input_ids)
#hidden_state = outputs.last_hidden_state

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5EncoderModel: ['decoder.block.3.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.4.layer.0.SelfAttention.k.weight', 'decoder.block.6.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.7.layer.0.SelfAttention.o.weight', 'decoder.block.7.layer.1.EncDecAttention.k.weight', 'decoder.final_layer_norm.weight', 'decoder.block.4.layer.0.SelfAttention.q.weight', 'decoder.block.7.layer.0.SelfAttention.v.weight', 'decoder.block.1.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.6.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.6.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.1.EncDecAttention.o.weight', 'decoder.block.4.layer.2.DenseReluDense.wo.weight', 'decoder.block.4.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.7.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.0.layer.0.SelfAttent

In [11]:
print(len(train_df[1200:]))
print(len(train_df[1200:len(train_df)]))

236
236


In [12]:
input_ids = []
#for frame in train_df['Text'][:400]:
lower = 1200
#upper = lower+400
upper = len(train_df)
for frame in train_df['Text'][lower:upper]:
    input_ids.append(tokenizer(frame, return_tensors="pt",truncation=True,max_length=64).input_ids)

outputs = []
for input_ in input_ids:
  outputs.append(model(input_))

hidden_state = []
for output_ in outputs:
  hidden_state.append(output_.last_hidden_state)

values = torch.zeros((400,512))
for i in range(len(hidden_state)):
  values[i] = (torch.mean(hidden_state[i][0],axis=0))

#save(folder_ + 'Data/' + 'data_400_800.npy', values.detach().numpy())

save(folder_ + 'Data/' + 'data_' + str(lower) + '_' + str(upper) + '.npy', values.detach().numpy())

In [10]:
len(input_ids)

600

In [13]:
lower = 0
upper = lower+400

In [29]:
li[3][236]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [22]:
lower = 0
upper = lower+400
li = []
for i in range(4):
  arr = np.load(folder_ + 'Data/' + 'data_' + str(lower) + '_' + str(upper) + '.npy')
  li.append(arr)
  lower+=400
  upper+=400
  if(lower == 1200):
    upper = 1436
  #break

In [53]:
final_arr = np.vstack((li[0],li[1],li[2],li[3]))
final_arr = final_arr[:1436]

In [54]:
final_arr.shape

(1436, 512)

In [56]:
final_arr[1435]

array([-5.26638143e-02, -1.54549867e-01,  1.86291840e-02, -1.83572739e-01,
       -7.87804574e-02,  5.09037301e-02,  2.03488126e-01,  3.10577769e-02,
       -2.82362122e-02, -4.99501117e-02,  2.41595507e-01, -1.40947133e-01,
        3.49488258e-02, -1.30859837e-01,  6.62851632e-02, -8.03611204e-02,
       -1.00439146e-01, -3.87492739e-02, -8.31745118e-02,  6.70289844e-02,
        2.66414620e-02,  1.25922114e-01, -8.44994038e-02,  5.71737066e-02,
       -3.76449794e-01,  2.24952493e-02, -8.02223682e-01,  2.64139920e-02,
        9.43697542e-02,  1.39501467e-01,  6.02034554e-02,  9.61242914e-02,
       -1.40072122e-01, -8.43710750e-02,  2.26025637e-02,  8.08138996e-02,
       -6.60696030e-02,  1.28506556e-01,  3.32188196e-02, -2.54327714e-01,
        1.76303789e-01, -9.96564031e-02, -1.18747167e-01,  9.18802321e-02,
        9.74093527e-02, -1.41530484e-01,  8.24178755e-02, -1.24160178e-01,
        5.84548041e-02, -2.65140980e-02, -1.03624135e-01, -2.03897849e-01,
       -2.53316015e-02,  

In [23]:
len(li)

4

In [21]:
print(upper)

400


In [19]:
li[0].shape

(400, 512)

In [16]:
arr[0]

array([ 7.08791763e-02, -2.02946827e-01, -1.56982150e-02, -2.34370932e-01,
       -1.09996378e-01,  2.73883110e-03,  1.57767594e-01, -1.20873973e-01,
        2.68560164e-02, -1.57486871e-01,  1.85399428e-01,  3.01670600e-02,
       -1.16110876e-01, -8.03325027e-02,  1.04789004e-01,  4.00744490e-02,
       -9.64827463e-03,  7.80769512e-02, -6.54094592e-02,  2.34272540e-01,
        9.03735012e-02,  1.47255048e-01, -2.97365904e-01, -3.12710879e-04,
       -4.81514990e-01,  3.51317227e-04, -7.38334060e-01, -2.45272331e-02,
        1.21216420e-02,  2.32545480e-01,  2.31464505e-02,  9.60460082e-02,
       -9.67335105e-02,  2.14932412e-02, -4.95644957e-02,  3.85352559e-02,
        6.06095046e-02,  2.17731535e-01,  1.56561822e-01, -2.69394934e-01,
        1.49119318e-01, -1.62100509e-01, -6.88991696e-02, -3.04846093e-02,
        3.67550701e-02, -1.43635780e-01,  6.77190796e-02,  8.12766477e-02,
       -9.13721398e-02, -6.80922195e-02, -1.08639903e-01, -1.19903527e-01,
       -3.17084938e-02,  

In [24]:
np.unique(train_df['Label']).shape[0]

20

In [57]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [58]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [59]:
print(Y_train.shape)
print(Y_train[0])

(1436, 20)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [60]:
print('Building model...')
model = Sequential()
model.add(Dense(1024, input_shape=(512,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(20))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#make one, takes in, double that amount, english and chichewa

Building model...


In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              525312    
                                                                 
 activation (Activation)     (None, 1024)              0         
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 20)                20500     
                                                                 
 activation_1 (Activation)   (None, 20)                0         
                                                                 
Total params: 545,812
Trainable params: 545,812
Non-trainable params: 0
_________________________________________________________________


In [63]:
X_train, X_test, y_train, y_test = train_test_split(
        final_arr, Y_train, test_size=0.3, random_state=42, stratify=Y_train)

In [64]:
X_train.shape

(1005, 512)

In [66]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [67]:
#train_dataset = tf.data.Dataset.from_tensor_slices((values.detach().numpy(), Y_train[:400]))
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [68]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [102]:
result = model.fit(train_dataset,
                    epochs=100, batch_size=32,
                    verbose=1)

Epoch 1/100
32/32 [==============================] - 0s 9ms/step - loss: 0.3742 - accuracy: 0.9214
Epoch 2/100
32/32 [==============================] - 0s 8ms/step - loss: 0.3455 - accuracy: 0.9274
Epoch 3/100
32/32 [==============================] - 0s 9ms/step - loss: 0.3394 - accuracy: 0.9284
Epoch 4/100
32/32 [==============================] - 0s 9ms/step - loss: 0.3128 - accuracy: 0.9393
Epoch 5/100
32/32 [==============================] - 0s 9ms/step - loss: 0.3075 - accuracy: 0.9463
Epoch 6/100
32/32 [==============================] - 0s 8ms/step - loss: 0.2985 - accuracy: 0.9453
Epoch 7/100
32/32 [==============================] - 0s 10ms/step - loss: 0.2816 - accuracy: 0.9423
Epoch 8/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2758 - accuracy: 0.9512
Epoch 9/100
32/32 [==============================] - 0s 8ms/step - loss: 0.2616 - accuracy: 0.9582
Epoch 10/100
32/32 [==============================] - 0s 8ms/step - loss: 0.2488 - accuracy: 0.9542
Epoch 1

In [105]:
# 20 epochs
model.evaluate(test_dataset)

14/14 [==============================] - 0s 3ms/step - loss: 3.4915 - accuracy: 0.4478


[3.4915149211883545, 0.4477958381175995]

In [103]:
ans = model.predict(test_dataset)

In [84]:
ans[0].shape

(20,)

In [86]:
y_test[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [104]:
print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1))))
print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1))),)

0.44779582366589327
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         7
           2       0.15      0.23      0.18        26
           3       0.29      0.15      0.20        13
           4       0.36      0.39      0.37        23
           5       0.00      0.00      0.00         2
           6       0.50      0.50      0.50        38
           7       0.45      0.54      0.49        41
           8       0.29      0.29      0.29         7
           9       1.00      0.50      0.67         4
          10       0.00      0.00      0.00         8
          11       0.61      0.64      0.62        84
          12       0.71      1.00      0.83        12
          13       0.43      0.52      0.47        44
          14       0.35      0.28      0.31        46
          15       0.39      0.35      0.37        40
          16       0.86      0.80      0.83        15
       

In [89]:
np_utils.to_categorical(np.argmax(ans[:2],axis=1))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1.]], dtype=float32)

In [70]:
# 10 epochs
model.evaluate(test_dataset)

14/14 [==============================] - 0s 3ms/step - loss: 1.8786 - accuracy: 0.4130


[1.8785860538482666, 0.41299304366111755]

In [ ]:
result = model.fit(values, Y_train[:400],
                    epochs=2, batch_size=32,
                    verbose=1)

TypeError: ignored

https://austingwalters.com/classify-sentences-via-a-multilayer-perceptron-mlp/
https://github.com/holbertonschool/deep-learning/blob/master/Class%20%233/Text%20Classification%20MLP%20Reuters%20Dataset.ipynb
https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2


Hidden state (embeddings):
https://github.com/huggingface/transformers/issues/1950